**[Day2]**  24.3.21.목

<small>2.12 (116p) ~ 5.2.1 (166p)<small>

In [1]:
import torch
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]]) 

3.12 텐서 직렬화

In [2]:
#points텐서를 ourpoints.t파일에 저장
torch.save(points, 'C:/Users/jw/Documents/study2/dlwpt-code/data/p1ch3/ourpoints.t')

<small> 파일 불러오는 3가지 방법

단, 포맷 호환 문제로 파이토치 외 다른 sw로 읽기 불가. 

=> 텐서를 호환 가능하게 저장하는 법 = h5py  <small>

In [3]:
with open('./dlwpt-code/data/p1ch3/ourpoints.t', 'wb') as f: #wb: 이진 쓰기 모드 #f : 파일을 열거나 생성후 읽고, 닫고 등 파일 작업하기 위해 생성된 객체. 
    torch.save(points, f)

In [4]:
points = torch.load('./dlwpt-code/data/p1ch3/ourpoints.t')

In [5]:
with open('./dlwpt-code/data/p1ch3/ourpoints.t', 'rb') as f:
    points = torch.load(f)

3.12.1 h5py로 HDF5 병렬화하기

In [6]:
import h5py

In [7]:
f = h5py.File('./dlwpt-code/data/p1ch3/ourpoints.hdf5', 'w') #w : 쓰기모드
dset = f.create_dataset('coords', data=points.numpy()) #데이터셋 이름: coords / 여기에 points의 넘파이 표현 저장
f.close() #open()으로 열었다가 작업 끝났으니 close()로 파일닫기~

In [13]:
#데이터셋의 일부 요소만 읽어오기
f = h5py.File('./dlwpt-code/data/p1ch3/ourpoints.hdf5', 'r') #읽기모드로 열기
dset = f['coords'] #방금 연 파일에서 'coords'라는 데이터셋임
last_points = dset[-2:] #끝에서 두번째~마지막 원소만 추출해 할당

In [58]:
last_points = torch.from_numpy(dset[-2:])  #torch.from_numpy : 넘파이를 텐서로 변환
f.close()

ValueError: Dset_id is not a dataset id (dset_id is not a dataset ID)

__4.1.2 이미지 로딩__

In [20]:
#imageio모듈을 사용한 PNG 이미지 로딩
import imageio

img_arr = imageio.imread('./dlwpt-code/data/p1ch4/image-dog/bobby.jpg')
img_arr.shape

C:\Users\jw\AppData\Local\Temp\ipykernel_22976\963339983.py:4: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img_arr = imageio.imread('./dlwpt-code/data/p1ch4/image-dog/bobby.jpg')


(720, 1280, 3)

4.1.3 레이아웃 변경하기

In [22]:
img = torch.from_numpy(img_arr)
out = img.permute(2, 0, 1)  #이미지 레이아웃을 토치에 맞게 변경(NCHW)


In [23]:
batch_size = 3
batch = torch.zeros(batch_size, 3, 256, 256, dtype=torch.uint8)


In [28]:
import os

data_dir = './dlwpt-code/data/p1ch4/image-cats/'
filenames = [name for name in os.listdir(data_dir)
             if os.path.splitext(name)[-1] == '.png']  #지정된 디렉토리에서 .png가진 모든 파일 이름 가져옴
for i, filename in enumerate(filenames): #enumerate = 해당 요소 반복순회
    img_arr = imageio.imread(os.path.join(data_dir, filename))
    img_t = torch.from_numpy(img_arr) #Numpy배열-> torch배열로 변환
    img_t = img_t.permute(2,0,1) #레이아웃 torch에 맞게 변경
    img_t = img_t[:3] #채널을 rgb로 제한]
    batch[i] = img_t #텐서를 배치 텐서에 추가함


C:\Users\jw\AppData\Local\Temp\ipykernel_22976\2754279828.py:7: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img_arr = imageio.imread(os.path.join(data_dir, filename))


4.1.4 데이터 정규화

In [29]:
#픽셀값을 0~1로 정규화
batch = batch.float() #텐서를 부동소수점(float) 자료형으로 변환
batch /= 255.0 #모든 픽셀값을 255로 나눔 

In [30]:
#배치의 각 채널에 대해 정규ㅜ화
n_channels = batch.shape[1] #배치의 채널 수 가져옴
for c in range(n_channels): #채널에 대해 반복
    mean = torch.mean(batch[:, c]) #현 채널의 평균 계산
    std = torch.std(batch[:, c]) #" 표준편차 계산
    batch[:, c] = (batch[:, c] - mean) / std #각 픽셀값을 -평균/편차

4.2.1 특수 포맷 로딩

In [33]:
#DICOM 파일 불러오기
import imageio

dir_path = "./dlwpt-code/data/p1ch4/volumetric-dicom/2-LUNG 3.0  B70f-04083" 
vol_arr = imageio.volread(dir_path, 'DICOM')
vol_arr.shape

Reading DICOM (examining files): 1/99 files (1.0%)

30/99 files (30.3%61/99 files (61.6%90/99 files (90.9%99/99 files (100.0%)
  Found 1 correct series.
Reading DICOM (loading data): 99/99  (100.0%)


(99, 512, 512)

In [35]:
#shape에서 사라진 채널 차원을 위한 공간 만들기
vol = torch.from_numpy(vol_arr).float()
vol = torch.unsqueeze(vol, 0)

vol.shape

torch.Size([1, 99, 512, 512])

4.3 테이블 데이터 표현하기

4.3.1 실세계 데이터셋 사용하기

4.3.2 와인 데이터를 텐서로 읽어오기

In [38]:
#텍스트 파일을 넘파이 배열로 변환
import csv
import numpy as np
wine_path = "./dlwpt-code/data/p1ch4/tabular-wine/winequality-white.csv"
wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=';',
                         skiprows=1) #q = quality~
wineq_numpy

array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  6.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]], dtype=float32)

In [39]:
#넘파이로 바꾼거 열어보기~
col_list = next(csv.reader(open(wine_path), delimiter=';'))

wineq_numpy.shape, col_list

((4898, 12),
 ['fixed acidity',
  'volatile acidity',
  'citric acid',
  'residual sugar',
  'chlorides',
  'free sulfur dioxide',
  'total sulfur dioxide',
  'density',
  'pH',
  'sulphates',
  'alcohol',
  'quality'])

In [40]:
#넘파이 배열을 파이토치 텐서로 바꾸기  
wineq = torch.from_numpy(wineq_numpy)

wineq.shape, wineq.dtype

(torch.Size([4898, 12]), torch.float32)

4.3.3 점수 표현하기

In [41]:
data = wineq[:, :-1] #모든행 + 마지막을 제외한 모든 열
data, data.shape

(tensor([[ 7.0000,  0.2700,  0.3600,  ...,  3.0000,  0.4500,  8.8000],
         [ 6.3000,  0.3000,  0.3400,  ...,  3.3000,  0.4900,  9.5000],
         [ 8.1000,  0.2800,  0.4000,  ...,  3.2600,  0.4400, 10.1000],
         ...,
         [ 6.5000,  0.2400,  0.1900,  ...,  2.9900,  0.4600,  9.4000],
         [ 5.5000,  0.2900,  0.3000,  ...,  3.3400,  0.3800, 12.8000],
         [ 6.0000,  0.2100,  0.3800,  ...,  3.2600,  0.3200, 11.8000]]),
 torch.Size([4898, 11]))

In [42]:
target = wineq[:, -1]
target, target.shape

(tensor([6., 6., 6.,  ..., 6., 7., 6.]), torch.Size([4898]))

In [44]:
#레이블 텐서의 target텐서를 전치하는 방법1.  : 정수 백터로 레이블 처리 => 연속값에 적당
target = wineq[:, -1].long()
target

tensor([6, 6, 6,  ..., 6, 7, 6])

4.3.4 원핫 인코딩

In [46]:
#레이블 텐서의 target텐서를 전치하는 방법1.  : 원핫 인코딩(한개만1 나머지는0) => 카테고리값에 적당
target_onehot = torch.zeros(target.shape[0], 10)

target_onehot.scatter_(1, target.unsqueeze(1), 1.0)


tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [47]:
target_unsqueezed = target.unsqueeze(1)
target_unsqueezed

tensor([[6],
        [6],
        [6],
        ...,
        [6],
        [7],
        [6]])

4.3.5 언제 카테고리화 할 것인가

In [48]:
#카테고리화 해보기

#각 열의평균 구하기
data_mean = torch.mean(data, dim=0) #dim=0 열을 따라 계산 / =1 행을 따라 계산
data_mean

tensor([6.8548e+00, 2.7824e-01, 3.3419e-01, 6.3914e+00, 4.5772e-02, 3.5308e+01,
        1.3836e+02, 9.9403e-01, 3.1883e+00, 4.8985e-01, 1.0514e+01])

In [49]:
#각 표준편차 구하기
data_var = torch.var(data, dim=0)
data_var

tensor([7.1211e-01, 1.0160e-02, 1.4646e-02, 2.5726e+01, 4.7733e-04, 2.8924e+02,
        1.8061e+03, 8.9455e-06, 2.2801e-02, 1.3025e-02, 1.5144e+00])

In [50]:
#정규화 : 열의 값 - 평균 / 표준편차
data_normalized = (data - data_mean) / torch.sqrt(data_var) #.sqrt : 주어진텐서 각 요소의 제곱근을 꼐산 = 분산을 표준편차로 바꾸기~~
data_normalized

tensor([[ 1.7208e-01, -8.1761e-02,  2.1326e-01,  ..., -1.2468e+00,
         -3.4915e-01, -1.3930e+00],
        [-6.5743e-01,  2.1587e-01,  4.7996e-02,  ...,  7.3995e-01,
          1.3422e-03, -8.2419e-01],
        [ 1.4756e+00,  1.7450e-02,  5.4378e-01,  ...,  4.7505e-01,
         -4.3677e-01, -3.3663e-01],
        ...,
        [-4.2043e-01, -3.7940e-01, -1.1915e+00,  ..., -1.3130e+00,
         -2.6153e-01, -9.0545e-01],
        [-1.6054e+00,  1.1666e-01, -2.8253e-01,  ...,  1.0049e+00,
         -9.6251e-01,  1.8574e+00],
        [-1.0129e+00, -6.7703e-01,  3.7852e-01,  ...,  4.7505e-01,
         -1.4882e+00,  1.0448e+00]])

4.3.6 임계값으로 찾기

In [51]:
#점수 3 이하인 열을 target에서 거르기
bad_indexes = target <= 3 #target에서 3이하인것만 선택
bad_indexes.shape, bad_indexes.dtype, bad_indexes.sum() #sum : 3이하가 true일때 true의 갯수 반환
#torch.bool : 열에서 true에 해당하는 행만 접근 가능~

(torch.Size([4898]), torch.bool, tensor(20))

In [53]:
bad_data = data[bad_indexes]
bad_data.shape

torch.Size([20, 11])

In [57]:
#와인 좋,보통,나쁨 카테고리로 나누기

#각 열에서 mean값 뽑기
bad_data = data[target <= 3]
mid_data = data[(target > 3) & (target < 7)]
good_data = data[target >= 7]

bad_mean = torch.mean(bad_data, dim=0)
mid_mean = torch.mean(mid_data, dim=0)
good_mean = torch.mean(good_data, dim=0)

for i, args in enumerate(zip(col_list, bad_mean, mid_mean, good_mean)):#열 목록, 해당열의 b,m,g 평균값 출력
    #i : 인덱스, args: zip으로 묶인 리스트의 요소들
    #enumerate() : 열 목록과 각 열의 평균값들을 반복적으로 가져옴
    #zip() : 여러 목록을 동시에 반복
    print('{:2} {:20} {:6.2f} {:6.2f} {:6.2f}'.format(i, *args)) 

 0 fixed acidity          7.60   6.89   6.73
 1 volatile acidity       0.33   0.28   0.27
 2 citric acid            0.34   0.34   0.33
 3 residual sugar         6.39   6.71   5.26
 4 chlorides              0.05   0.05   0.04
 5 free sulfur dioxide   53.33  35.42  34.55
 6 total sulfur dioxide 170.60 141.83 125.25
 7 density                0.99   0.99   0.99
 8 pH                     3.19   3.18   3.22
 9 sulphates              0.47   0.49   0.50
10 alcohol               10.34  10.26  11.42


In [61]:
#이산화황의 중앙점을 임계값으로. 그 이하만 출력
total_sulfur_threshold = 141.83
total_sulfur_data = data[:, 6]
predicted_indexes = torch.lt(total_sulfur_data, total_sulfur_threshold)

predicted_indexes.shape, predicted_indexes.dtype, predicted_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(2727))

In [65]:
#실제로 좋은 와인의 인덱스 뽑기
actual_indexes = target > 5

actual_indexes.shape, actual_indexes.dtype, actual_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(3258))

In [66]:
#실제 순위와 예측 순위 확인
n_matches = torch.sum(actual_indexes & predicted_indexes).item() #실제와 예측이 동시에 발생한 경우를 bool로
n_predicted = torch.sum(predicted_indexes).item()
n_actual = torch.sum(actual_indexes).item()

n_matches, n_matches / n_predicted, n_matches / n_actual #겹친거, 예측 결과, 실제예측결과

(2018, 0.74000733406674, 0.6193984039287906)


4.4 시계열 데이터 다루기

4.4.1 시간 차원 더하기

In [75]:
file_path = "./dlwpt-code/data/p1ch4/bike-sharing-dataset/hour-fixed.csv"
with open(file_path,  'r') as file:
    data = file.read()

print(data)

instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0,3,13,16
2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.8,0,8,32,40
3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.8,0,5,27,32
4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0,3,10,13
5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0,0,1,1
6,2011-01-01,1,0,1,5,0,6,0,2,0.24,0.2576,0.75,0.0896,0,1,1
7,2011-01-01,1,0,1,6,0,6,0,1,0.22,0.2727,0.8,0,2,0,2
8,2011-01-01,1,0,1,7,0,6,0,1,0.2,0.2576,0.86,0,1,2,3
9,2011-01-01,1,0,1,8,0,6,0,1,0.24,0.2879,0.75,0,1,7,8
10,2011-01-01,1,0,1,9,0,6,0,1,0.32,0.3485,0.76,0,8,6,14
11,2011-01-01,1,0,1,10,0,6,0,1,0.38,0.3939,0.76,0.2537,12,24,36
12,2011-01-01,1,0,1,11,0,6,0,1,0.36,0.3333,0.81,0.2836,26,30,56
13,2011-01-01,1,0,1,12,0,6,0,1,0.42,0.4242,0.77,0.2836,29,55,84
14,2011-01-01,1,0,1,13,0,6,0,2,0.46,0.4545,0.72,0.2985,47,47,94
15,2011-01-01,1,0,1,14,0,6,0,2,0.46,0.4545,0.72,0.2836,35,71,106
16,2011-01

In [69]:
bikes_numpy = np.loadtxt(
    "./dlwpt-code/data/p1ch4/bike-sharing-dataset/hour-fixed.csv",
    dtype=np.float32,
    delimiter=",", #쉼표로 각열 구분
    skiprows=1, #첫행 건너뛰기
    converters={1: lambda x: float(x[8:10])}) #특정열(여기선1) 반환. x로 받아서 8~9는 실수로 변환
bikes = torch.from_numpy(bikes_numpy) #csv to numpy
bikes

tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 3.0000e+00, 1.3000e+01,
         1.6000e+01],
        [2.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 8.0000e+00, 3.2000e+01,
         4.0000e+01],
        [3.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 5.0000e+00, 2.7000e+01,
         3.2000e+01],
        ...,
        [1.7377e+04, 3.1000e+01, 1.0000e+00,  ..., 7.0000e+00, 8.3000e+01,
         9.0000e+01],
        [1.7378e+04, 3.1000e+01, 1.0000e+00,  ..., 1.3000e+01, 4.8000e+01,
         6.1000e+01],
        [1.7379e+04, 3.1000e+01, 1.0000e+00,  ..., 1.2000e+01, 3.7000e+01,
         4.9000e+01]])

4.4.2 시간 단위로 데이터 만들기

In [76]:
#일별 데이터셋 구하기 위해, 동일 텐서를 24시간 배치로 보기
#bikes 텐서의 차원, 스트라이드 확인
bikes.shape, bikes.stride() #17420시간, 17개열 #stride: 메모리상 이동량 (104p)

(torch.Size([17520, 17]), (17, 1))

In [78]:
#데이터를 일자, 시간, 17개 열 : 3개 축으로 만들기
daily_bikes = bikes.view(-1, 24, bikes.shape[1]) #shape[1] : 텐서열. 17개. 
#**view: 저장 공간 바꾸지 않고, 차원수나 스트라이드 정보 바꾼 새 텐서 반환. (): 새로받을텐서가 가질 차원정보
#-1 = placeholder : 남은 차원과 거기있는 요소 다 합친것
#
daily_bikes.shape, daily_bikes.stride() #shape 출력: (일수, 24, 원래dataset의 열 수 )
# 기존 stride: 한 차원(=한시간)이동시 17칸 이동해야함. => 여기선 24시간 이동해야 하니까 24 * 17 = 408

(torch.Size([730, 24, 17]), (408, 17, 1))

4.4.3 훈련 준비

In [80]:
#날씨 : 1 좋음 4 안좋음
first_day = bikes[:24].long() #첫째날에서, 열인덱스가 0인걸 선택하라. => 시간당 날씨 확인
weather_onehot = torch.zeros(first_day.shape[0], 4) #초기화된 크기: 모든요소 0, 날씨종류 4개니까
first_day[:, 9] #모든행 +9번째 열 선택

tensor([1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2])

In [81]:
#날씨 수준에 따라 행렬을 원핫 인코딩으로 변환
weather_onehot.scatter_(
    dim=1, #열
    index=first_day[:,9].unsqueeze(1).long() -1, #unsqueeze: 열을 열벡터로 만듦 / long() : 정수형으로 변환 #각인덱스 -1
    value=1.0) #인코딩된 위치에 1.0 할당

tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]])

In [82]:
torch.cat((bikes[:24], weather_onehot), 1)[:1] #(24행 선택, 원핫실행), 얘네 둘을 열방향으로(세로)이어붙임
#[:1]: 새로운 텐서에서 첫행만 선택
#결과: 대여량.날씨                                      

tensor([[ 1.0000,  1.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  6.0000,
          0.0000,  1.0000,  0.2400,  0.2879,  0.8100,  0.0000,  3.0000, 13.0000,
         16.0000,  1.0000,  0.0000,  0.0000,  0.0000]])

In [83]:
#daily_bikes(형태 : B, L(24), C)
#값0인텐서만들기
daily_weather_onehot = torch.zeros(daily_bikes.shape[0], 4,
                                   daily_bikes.shape[2])
daily_weather_onehot.shape
#3D 텐서임. [0]: 일일 자전거수, 4: 날씨종류, [2]: 시간당 자전거수

torch.Size([730, 4, 17])

In [88]:
daily_weather_onehot.scatter_(
    1, daily_bikes[:,9,:].long().unsqueeze(1) - 1, 1.0)
daily_weather_onehot.shape


RuntimeError: index 9 is out of bounds for dimension 1 with size 4

In [87]:
#C차원 따라 원핫 인코딩 병합
daily_bikes = torch.cat((daily_bikes, daily_weather_onehot), dim=1)

In [90]:
#<날씨 상태를 연속변수로 보고 처리하는 다른 방법:>
#1안. 값을 0.0, 1.0으로 변환
daily_bikes[:, 9, :] = (daily_bikes[:, 9, :] -1.0 /3.0)

In [94]:
#값의 범위를 0.0~1.0으로 매핑
temp = daily_bikes[:, 10, :]
temp_min = torch.min(temp)
temp_max = torch.max(temp)
daily_bikes[:, 10, :] = ((daily_bikes[:, 10, :] - temp_min)
                         / (temp_max - temp_min))

In [95]:
#2안. 평균0, 단위 표준편차 가지는 값으로 바뀜. 가우시안 분포로 그리면 68%는 -1 ~1사이에 분포함. 
temp = daily_bikes[:, 10, :]
daily_bikes[:, 10, :] = ((daily_bikes[:, 10, :] - torch.mean(temp)) 
                         / torch.std(temp))
daily_bikes[:, 10, :]

tensor([[-0.2197, -0.2240, -0.2240,  ..., -0.2193, -0.2142, -0.2092],
        [-0.2100, -0.2235, -0.2240,  ..., -0.2214, -0.2050, -0.2020],
        [-0.2007, -0.2231, -0.2240,  ..., -0.2197, -0.2104, -0.2058],
        ...,
        [ 7.0838, -0.2121, -0.2240,  ..., -0.2218, -0.2096, -0.2071],
        [ 7.0939, -0.2117, -0.2240,  ..., -0.2113, -0.1860, -0.1729],
        [ 7.1041, -0.2113, -0.2240,  ..., -0.2104, -0.1877, -0.1737]])

4.5 텍스트 표현하기

4.5.1 텍스트를 숫자로 변환하기

In [101]:
with open("./dlwpt-code/data/p1ch4/jane-austen/1342-0.txt", encoding='utf8') as f:
    text = f.read()

4.5.2 문자 원핫 인코딩

In [102]:
#텍스트를 행으로 나누고 몇 줄 가져오기
lines = text.split('\n') #(\n : 줄바꿈) 을 기준으로 나눈다~
line = lines[200] #201번째 인덱스 가져옴
line

'“Impossible, Mr. Bennet, impossible, when I am not acquainted with him'

In [103]:
letter_t = torch.zeros(len(line), 128) #ascii제한인 128로 하드코딩(직접 코드에 값을 넣음) #len: 수 반환
letter_t.shape #행마다 원핫인코딩된 문자 하나 있음

torch.Size([70, 128])

In [105]:
#행이 일치하는 문자 표현하도록 정확한 위치에 값으로 1 기록
for i, letter in enumerate(line.lower().strip()):  #lower: 소문자 변환 #strip() : 문자열 양쪽 공백 제거    
    letter_index = ord(letter) if ord(letter) < 128 else 0 #128 미만인 경우만 사용. 아님 0 #ord : 문자의 유니코드 코드 포인트(정수)를 반환
    letter_t[i][letter_index] = 1 #텐서 letter_t : 행은 텍스트의 각 문자 / 열은 아스키코드 인덱스 1, 나머지0 
    #i는 문자위치, letterindex는 아스키 코드

4.5.3 모든 단어를 원핫 인코딩하기

In [107]:
def clean_words(input_str):
    punctuation = '.,;:"!?""_-' #제거할 문자
    word_list = input_str.lower().replace('\n',' ').split() #split() : 공백을 기준으로 문자열을 단어로 분할
    word_list = [word.strip(punctuation) for word in word_list] #strip() : 앞뒤제거
    return word_list

words_in_line = clean_words(line)
line, words_in_line


('“Impossible, Mr. Bennet, impossible, when I am not acquainted with him',
 ['“impossible',
  'mr',
  'bennet',
  'impossible',
  'when',
  'i',
  'am',
  'not',
  'acquainted',
  'with',
  'him'])

In [109]:
#인코딩에서 단어를 인덱스로 매핑
word_list = sorted(set(clean_words(text))) #set: 중복되지 않는 단어들의 집합 생성 #sorted(): 집합을 알파벳순으로 정렬
word2index_dict = {word: i for (i, word) in enumerate(word_list)} #enumerate(): 정렬된 단어목록의 각 단어-인덱스 쌍 생성
#word2index_dict: 단어가 key, 정수가 값인 dictionary

len(word2index_dict), word2index_dict['impossible'] #고유 단어수, impossible이라는 단어의 인덱스

(8484, 3828)

In [110]:
#빈벡터 생성. 각 단어의 원핫인코딩값 부여
word_t = torch.zeros(len(words_in_line), len(word2index_dict))
for i, word in enumerate(words_in_line):
    word_index = word2index_dict[word]
    word_t[i][word_index] = 1 #인덱스에 해당하는 열을 1로 설정
    print('{:2} {:4} {}'.format(i, word_index, word)) #{}: 자릿수 
    
print(word_t.shape)


 0 8324 “impossible
 1 4905 mr
 2  891 bennet
 3 3828 impossible
 4 8017 when
 5 3740 i
 6  445 am
 7 5054 not
 8  247 acquainted
 9 8094 with
10 3619 him
torch.Size([11, 8484])


5.2 학습은 파라미터 추정에 불과하다

5.2.1 온도 문제

5.2.2 데이터 수집

In [111]:
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0] #섭씨
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4] #모르는 단위의 값
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)

5.2.3 데이터 시각화

5.2.4 먼저 선형 모델을 골라 시도해보기